In [ ]:
import requests
import json

In [ ]:
GOOGLE_MAPS_KEY = ""

In [ ]:
# import requests

# vertrek = 'Prins Bernhardplein 175'
# bestemming = 'Amsterdam Science Park'

# url = "https://maps.googleapis.com/maps/api/directions/json?origin={origin}&destination={destination}&mode=transit&key=".format(origin=vertrek, destination=bestemming) + GOOGLE_MAPS_KEY

# payload={}
# headers = {}

# response = requests.request("GET", url, headers=headers, data=payload)

# print(response.text)

In [ ]:
import requests

ORIG = 'Prins Bernhardplein 175'
DEST = 'Amsterdam Science Park'

url = "https://maps.googleapis.com/maps/api/directions/json?origin={origin}&destination={destination}&mode=transit&key=".format(origin=ORIG, destination=DEST) + GOOGLE_MAPS_KEY

payload={}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)

response = eval(response.text)

departure_time = response['routes'][0]['legs'][0]['departure_time']['text']
print(departure_time)

arrival_time = response['routes'][0]['legs'][0]['arrival_time']['text']
print(arrival_time)

start_address = response['routes'][0]['legs'][0]['start_address']
print(start_address)

end_address = response['routes'][0]['legs'][0]['end_address']
print(end_address)

end_loc = response['routes'][0]['legs'][0]['end_location']
print(end_loc)

duration = response['routes'][0]['legs'][0]['duration']['text']
print(duration)

legs = []
for i in response['routes'][0]['legs'][0]['steps']:
    try:
        legs.append((i['distance']['text'], i['duration']['text'], i['travel_mode'], i['transit_details']['line']['vehicle']['type'], i['transit_details']['headsign'], i['transit_details']['line']['short_name'], i['transit_details']['num_stops']))
    except:
        legs.append((i['distance']['text'], i['duration']['text'], i['travel_mode']))
print(legs)



In [ ]:
import pandas as pd
df = pd.read_csv('lijn40_ordered.csv')
df

In [ ]:
DARKSKY_KEY = ""

In [ ]:
last_departure_stop = 'Amsterdam, Amstelstation'
last_departure_time = '4:51pm'
last_arrival_stop = 'Amsterdam, Science Park Terra'
last_arrival_time = '5:01pm'
final_destination = DEST
final_arrival_time = '5:02pm'

def get_weather(longitude,latitude):
#Gets a json_object with the current weather on the given coordinate
    URL = "https://api.darksky.net/forecast/" + DARKSKY_KEY + "/" + str(latitude) + "," + str(longitude) +"?exclude=daily,alerts"
    r = requests.get(URL)
    json_str = r.content.decode("utf8").replace("'", '"')
    json_object = json.loads(json_str)
    return json_object

def go_walk(last_departure_stop, last_departure_time, last_arrival_stop, last_arrival_time, final_destination, final_arrival_time):
    
    # read to use the csv of line 40
    df = pd.read_csv('lijn40_ordered.csv')
    
    # the orderlocation of the busstop where the last leg starts
    A = df[df['haltenaam'] == last_departure_stop].index.values[0]
    
    # the order location of the arrival stop
    B = df[df['haltenaam'] == last_arrival_stop].index.values[0]
    
    # get all stops in between the start and arrival of the last leg and order them correctly
    if A < B:
        stoplist = df[A:B+1]['haltenaam']
    if B < A:
        stoplist = df[B:A+1]['haltenaam'][::-1]
    
    # get the walking time and distance from each stop to the final destination
    results = []
    for stop in stoplist:
        url = "https://maps.googleapis.com/maps/api/directions/json?origin={origin}&destination={destination}&mode=walking&key=".format(origin=stop, destination=final_destination) + GOOGLE_MAPS_KEY
        payload={}
        headers = {}
        response = requests.request("GET", url, headers=headers, data=payload)
        response = eval(response.text)
        
        # get duration and distance
        duration = response['routes'][0]['legs'][0]['duration']['text']
        distance = response['routes'][0]['legs'][0]['distance']['text']
        
        # get start address (for control if it picked the right lcoation)
        start_address = response['routes'][0]['legs'][0]['start_address']
        
        # order results that are returened
        results.append([start_address, duration, distance])
        
        # get lng and lat of end location for weather
        end = response['routes'][0]['legs'][0]['end_location']
        
        # get weather and oder results
        weather = get_weather(end['lng'], end['lat'])
        weather_result = [weather['currently']['summary'], weather['currently']['precipProbability'], weather['currently']['temperature']]
    
    return results, weather_result
    
go_walk(last_departure_stop, last_departure_time, last_arrival_stop, last_arrival_time, final_destination, final_arrival_time)